In [1]:
import numpy as np
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.optimize import curve_fit

In [12]:
path0p01 = 'idms_0p01/nuevoScan/'
path0p05 = 'idms_0p05/nuevoScan/'
path0p1 = 'idms_0p1/nuevoScan/'
path0p1_2 = 'idms_0p1/nuevoScan2/'
path0p2 = 'idms_0p2/nuevoScan/'
path0p2_2 = 'idms_0p2/nuevoScan2/'
path0p4 = 'idms_0p4/nuevoScan/'
path0p4_2 = 'idms_0p4/nuevoScan2/'
pathImagenes = 'imagenes_idms/'
pathLimites = 'limitesidms/'

In [13]:
def calc_mchi2(mchi1, ma, gx, f):
    vphi = ma / (2*gx)
    return mchi1 + 2*f*vphi

def calc_lim_ms2(alphad, ma):
    return ma**2 / (4*np.sqrt(np.pi)* alphad)


def calc_values(df_dat):
    mx1 = df_dat.column('Mx1')
    yf = df_dat.column('yf')
    vphi = df_dat.column('MAp') / (2*df_dat.column('gX'))
    mx2 = mx1 + 2*yf*vphi
    df_dat['Mx2'] =  mx2
    df_dat['delta'] = df_dat.column('Mx2') - df_dat.column('Mx1')
    df_dat['alphad'] = (df_dat.column('gX')**2)/(4*np.pi)
    df_dat['alphaf'] = (df_dat.column('yf')**2)/(4*np.pi)
    #df_dat['ratio_ms2_map'] = df_dat.column('Ms2')**2 / calc_lim_ms2(df_dat.column('alphad'),df_dat.column('MAp'))
    df_dat['mass_relative'] = df_dat.column('delta') / df_dat.column('Mx1')
    return df_dat

def convertir_elementos(df_):
    # Crear una copia del DataFrame original
    df = df_.copy() 
        
    # Convertir y redondear las columnas numéricas
    df['Mchi1'] = df['Mchi1'] * 1e3 
    df['MAp'] = df['MAp'] * 1e3
    df['mphi'] = df['mphi'] * 1e3
    
    # Renombrar las columnas
    df = df.relabel('Mchi1', 'Mx1')
    df = df.relabel('Mchi2', 'Mx2')
    df = df.relabel('mphi', 'Ms2')
    df = df.relabel('ff', 'yf')
    df = df.relabel('angle', 'st')
    df = calc_values(df)
    return df 


def cargado_archivos(archivos):
    """
    Combina múltiples archivos CSV en una única tabla, manejando archivos vacíos o inexistentes.

    Args:
        archivos (list): Lista de rutas de archivos CSV.

    Returns:
        Table: Tabla combinada. Si no hay archivos válidos, retorna una tabla vacía.
    """
    datos = None  # Inicializamos como None para manejar el primer archivo válido

    for archivo in archivos:
        try:
            tabla_temp = Table.read_table(archivo)
            if tabla_temp.num_rows > 0:
                if datos is None:  # Primer archivo válido
                    datos = tabla_temp
                else:
                    datos.append(tabla_temp)  # Append a la tabla existente
        except FileNotFoundError:
            print(f"Error: {archivo} no existe.")
        except Exception as e:  # Otros errores (ej: formato incorrecto)
            print(f"Error al cargar {archivo}: {str(e)}")

    return Table() if datos is None else datos  # Retorna tabla vacía si no hay datos


#Los valores reportados por Planck
val_exp = 0.12 #Valor observado
error = 0.001 #Error reportado por Planck 
#Queremos que el valor sea calculado a 3 sigma, o sea, 3 veces el error reportado por Planck 
sigma = 3
val_por = sigma *  error 
val_min = round(val_exp - val_por,3)
val_max = round(val_exp + val_por,3)

def omega(df_):
    global val_min, val_max
    df = df_.copy()
    df['Densidad reliquia'] = np.round(df.column('Densidad reliquia'),3)
    return df.where('Densidad reliquia', are.between_or_equal_to(val_min,val_max))
    
def graficar_densidad_reliquia(df):
    plt.figure(figsize=(7, 5))  # Ancho mayor para 3 gráficos


    #plt.plot(df_5.column('Mx1'), df_5.column('Densidad reliquia'),'.',color='blue',alpha=0.5)
    plt.plot(df.column('Mx1'), df.column('Densidad reliquia'), 
             '.', color='blue', alpha=0.7, markersize=8)
    plt.axhline(y=val_min, color='red', linestyle='--', linewidth=1)
    plt.axhline(y=val_max, color='red', linestyle='--', linewidth=1)
    plt.ylabel(r'$\Omega h^2$', fontsize=14)
    plt.xlabel(r'$M_{\chi_{1}}$ (GeV)', fontsize=14)
    plt.ylim(0.115, 0.125)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.xscale('log')
    plt.yscale('linear')
    
    
    plt.tight_layout()

def graficar_epsilon(df):
    plt.figure(figsize=(10,7))
    plt.plot(df.column('Mx1'), df.column('epsilon') ,'.',color='blue',alpha=0.5)
    plt.xscale('log')
    plt.yscale('log')
    plt.ylabel(r'$\epsilon$',size=20)
    plt.xlabel(r'$M_{\chi_{1}}$ (MeV)',size=20)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    #plt.legend(fontsize=12, framealpha=0.9, loc='best')
    #plt.title(r'$\Delta \geq 2m_{e}$',size=20)
    plt.xlim(0.9,1e3+100)


def graficar_y(df_):
    df = df_.copy()
    alpha = df.column('alphad')[0]
    ratio = round((df.column('Mx1') / df.column('MAp')).item(0),3)
    factor = alpha * ratio**4 
    plt.figure(figsize=(10,7))
    plt.plot(df.column('Mx1'), df.column('epsilon')**2 * factor ,'.',color='blue',alpha=0.5)
    plt.xscale('log')
    plt.yscale('log')
    plt.ylabel(r'$y = \alpha_{D}\epsilon^2(M_{\chi_{1}}/M_{A^\prime})^{4}$',size=20)
    plt.xlabel(r'$M_{\chi_{1}}$ (MeV)',size=20)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    #plt.legend(fontsize=12, framealpha=0.9, loc='best')
    #plt.title(r'$\Delta \geq 2m_{e}$',size=20)
    plt.xlim(0.9,1e3+100)


def calc_values_idm(df_dat):
    mx1 = df_dat.column('Mx1')
    mx2 = mx1 + df_dat.column('deltamass')
    df_dat['Mx2'] =  mx2
    df_dat['delta'] = df_dat.column('Mx2') - df_dat.column('Mx1')
    df_dat['alphad'] = (df_dat.column('gX')**2)/(4*np.pi)
    df_dat['mass_relative'] = df_dat.column('delta') / df_dat.column('Mx1')
    return df_dat

def convertir_elementos_idm(df_):
    # Crear una copia del DataFrame original
    df = df_.copy() 
        
    # Convertir y redondear las columnas numéricas
    df['Mchi1'] = df['Mchi1'] * 1e3 
    df['MAp'] = df['MAp'] * 1e3
    #df['mphi'] = df['mphi'] * 1e3
    
    # Renombrar las columnas
    df = df.relabel('Mchi1', 'Mx1')
    df = df.relabel('Mchi2', 'Mx2')
    df = calc_values_idm(df)
    return df 


def limite_masa_higgs_foton_oscuro(df_):
    df = df_.copy()
    factor = df.column('alphad').item(0)
    #print(factor)
    mask = df.column('Ms2')**2 <= df.column('MAp')**2 / factor
    df_filtrado = df.where(mask)
    return df_filtrado


def extrapolacion(data, label1,label2, degree=5, extension_factor=0.2):
    mass = data.column(label1)
    y = data.column(label2)
    
    coefficients = np.polyfit(mass, y, degree)
    polynomial = np.poly1d(coefficients)
    
    log_min = np.log10(min(mass) * (1 - extension_factor))
    log_max = np.log10(max(mass) * (1 + extension_factor))
    
    mass_extended = np.logspace(log_min, log_max, 500)
    y_extrapolated = polynomial(mass_extended)
    return mass_extended, y_extrapolated


def restriccion_2(mx1,mphi,alphaf):
    return (1/4)*alphaf**2*mx1 < mphi

def mask_2(df_):
    mask = [(1/4)*alphaf**2 * mx1 < mphi 
        for mx1, mphi, alphaf in zip(
            df_['Mx1'], 
            df_['Ms2'], 
            df_['alphaf'])]
    return mask

In [15]:
#analisis_random_Delta0p1_alpha0p5_R5_8.csv


archivos1 = [f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_1.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_2.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_3.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_4.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_5.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_6.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_7.csv',
            f'{path0p1_2}analisis_random_Delta0p1_alpha0p5_R5_8.csv']

df_0p1_R5 = cargado_archivos(archivos1)
df_0p1_R5 = convertir_elementos(df_0p1_R5)
df_omega_0p1_R5 = omega(df_0p1_R5)

df_omega_0p1_R5.to_csv(f'{path0p1}Delta0p1_alpha0p5_R5_omega2.csv')

In [10]:
archivos1 = [f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_1.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_2.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_3.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_4.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_5.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_6.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_7.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R5_8.csv']

df_0p2_R5 = cargado_archivos(archivos1)

df_0p2_R5 = convertir_elementos(df_0p2_R5)
#df_0p01_R3.show(3)
df_omega_0p2_R5 = omega(df_0p2_R5)

df_omega_0p2_R5.to_csv(f'{path0p2}Delta0p2_alpha0p5_R5_omega2.csv')


archivos1 = [f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_1.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_2.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_3.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_4.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_5.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_6.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_7.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_8.csv']

df_0p2_R8 = cargado_archivos(archivos1)

df_0p2_R8 = convertir_elementos(df_0p2_R8)
#df_0p01_R3.show(3)
df_omega_0p2_R8 = omega(df_0p2_R8)

df_omega_0p2_R8.to_csv(f'{path0p2}Delta0p2_alpha0p5_R8_omega2.csv')

archivos1 = [f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_1.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_2.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_3.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_4.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_5.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_6.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_7.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R10_8.csv']

df_0p2_R10 = cargado_archivos(archivos1)
df_0p2_R10 = convertir_elementos(df_0p2_R10)
df_omega_0p2_R10 = omega(df_0p2_R10)

df_omega_0p2_R10.to_csv(f'{path0p2}Delta0p2_alpha0p5_R10_omega2.csv')

In [11]:
archivos1 = [f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_1.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_2.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_3.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_4.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_5.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_6.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_7.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R5_8.csv']

df_0p4_R5 = cargado_archivos(archivos1)
df_0p4_R5 = convertir_elementos(df_0p4_R5)
df_omega_0p4_R5 = omega(df_0p4_R5)

df_omega_0p4_R5.to_csv(f'{path0p4}Delta0p4_alpha0p5_R5_omega2.csv')


archivos1 = [f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_1.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_2.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_3.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_4.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_5.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_6.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_7.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_8.csv']

df_0p4_R8 = cargado_archivos(archivos1)
df_0p4_R8 = convertir_elementos(df_0p4_R8)
df_omega_0p4_R8 = omega(df_0p4_R8)

df_omega_0p4_R8.to_csv(f'{path0p4}Delta0p4_alpha0p5_R8_omega2.csv')

archivos1 = [f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_1.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_2.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_3.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_4.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_5.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_6.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_7.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R10_8.csv']

df_0p4_R10 = cargado_archivos(archivos1)
df_0p4_R10 = convertir_elementos(df_0p4_R10)
df_omega_0p4_R10 = omega(df_0p4_R10)

df_omega_0p4_R10.to_csv(f'{path0p4}Delta0p4_alpha0p5_R10_omega2.csv')

In [4]:
archivos1 = [f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_1.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_2.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_3.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_4.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_5.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_6.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_7.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R3_8.csv']

df_0p2_R3 = cargado_archivos(archivos1)

df_0p2_R3 = convertir_elementos(df_0p2_R3)
#df_0p01_R3.show(3)
df_omega_0p2_R3 = omega(df_0p2_R3)

df_omega_0p2_R3.to_csv(f'{path0p2}Delta0p2_alpha0p5_R3_omega.csv')



archivos1 = [f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_1.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_2.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_3.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_4.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_5.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_6.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_7.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R5_8.csv']

df_0p2_R5 = cargado_archivos(archivos1)

df_0p2_R5 = convertir_elementos(df_0p2_R5)
#df_0p01_R5.show(3)
df_omega_0p2_R5 = omega(df_0p2_R5)
df_omega_0p2_R5.to_csv(f'{path0p2}Delta0p2_alpha0p5_R5_omega.csv')

archivos1 = [f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_1.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_2.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_3.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_4.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_5.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_6.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_7.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R8_8.csv']

df_0p2_R8 = cargado_archivos(archivos1)

df_0p2_R8 = convertir_elementos(df_0p2_R8)
#df_0p01_R8.show(3)
df_omega_0p2_R8 = omega(df_0p2_R8)
df_omega_0p2_R8.to_csv(f'{path0p2}Delta0p2_alpha0p5_R8_omega.csv')

archivos1 = [f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_1.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_2.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_3.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_4.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_5.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_6.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_7.csv',
            f'{path0p2}analisis_random_Delta0p2_alpha0p5_R10_8.csv']

df_0p2_R10 = cargado_archivos(archivos1)

df_0p2_R10 = convertir_elementos(df_0p2_R10)
#df_0p01_R10.show(3)
df_omega_0p2_R10 = omega(df_0p2_R10)
df_omega_0p2_R10.to_csv(f'{path0p2}Delta0p2_alpha0p5_R10_omega.csv')

In [5]:
archivos1 = [f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_1.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_2.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_3.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_4.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_5.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_6.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_7.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R3_8.csv']

df_0p4_R3 = cargado_archivos(archivos1)

df_0p4_R3 = convertir_elementos(df_0p4_R3)
#df_0p01_R3.show(3)
df_omega_0p4_R3 = omega(df_0p4_R3)

df_omega_0p4_R3.to_csv(f'{path0p4}Delta0p2_alpha0p5_R3_omega.csv')



archivos1 = [f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_1.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_2.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_3.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_4.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_5.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_6.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_7.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R5_8.csv']

df_0p4_R5 = cargado_archivos(archivos1)

df_0p4_R5 = convertir_elementos(df_0p4_R5)
#df_0p01_R5.show(3)
df_omega_0p4_R5 = omega(df_0p4_R5)
df_omega_0p4_R5.to_csv(f'{path0p4}Delta0p4_alpha0p5_R5_omega.csv')

archivos1 = [f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_1.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_2.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_3.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_4.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_5.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_6.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_7.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R8_8.csv']

df_0p4_R8 = cargado_archivos(archivos1)

df_0p4_R8 = convertir_elementos(df_0p4_R8)
#df_0p01_R8.show(3)
df_omega_0p4_R8 = omega(df_0p4_R8)
df_omega_0p4_R8.to_csv(f'{path0p4}Delta0p4_alpha0p5_R8_omega.csv')

archivos1 = [f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_1.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_2.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_3.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_4.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_5.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_6.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_7.csv',
            f'{path0p4}analisis_random_Delta0p4_alpha0p5_R10_8.csv']

df_0p4_R10 = cargado_archivos(archivos1)

df_0p4_R10 = convertir_elementos(df_0p4_R10)
#df_0p01_R10.show(3)
df_omega_0p4_R10 = omega(df_0p4_R10)
df_omega_0p4_R10.to_csv(f'{path0p4}Delta0p2_alpha0p5_R10_omega.csv')

In [10]:
path0p2_2 = 'idms_0p2/nuevoScan2/'
path0p4_2 = 'idms_0p4/nuevoScan2/'

archivos1 = [f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_1.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_2.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_3.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_4.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_5.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_6.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_7.csv',
            f'{path0p2_2}analisis_random_Delta0p2_alpha0p5_R8_8.csv']

df_0p2_R8 = cargado_archivos(archivos1)

df_0p2_R8 = convertir_elementos(df_0p2_R8)
df_omega_0p2_R8 = omega(df_0p2_R8)
df_omega_0p2_R8.to_csv(f'{path0p2_2}Delta0p2_alpha0p5_R8_omega.csv')

archivos1 = [f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_1.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_2.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_3.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_4.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_5.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_6.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_7.csv',
            f'{path0p4_2}analisis_random_Delta0p4_alpha0p5_R8_8.csv']

df_0p4_R8 = cargado_archivos(archivos1)

df_0p4_R8 = convertir_elementos(df_0p4_R8)
df_omega_0p4_R8 = omega(df_0p4_R8)
df_omega_0p4_R8.to_csv(f'{path0p4_2}Delta0p4_alpha0p5_R8_omega.csv')

In [8]:
archivos1 = [f'{path0p01}analisis_random_Delta0p011.csv',
            f'{path0p01}analisis_random_Delta0p012.csv',
            f'{path0p01}analisis_random_Delta0p013.csv',
            f'{path0p01}analisis_random_Delta0p014.csv',
            f'{path0p01}analisis_random_Delta0p015.csv',
            f'{path0p01}analisis_random_Delta0p016.csv',
            f'{path0p01}analisis_random_Delta0p017.csv',
            f'{path0p01}analisis_random_Delta0p018.csv']

df_0p01_R3 = cargado_archivos(archivos1)

df_0p01_R3 = convertir_elementos(df_0p01_R3)
#df_0p01_R3.show(3)
df_omega_0p01_R3 = omega(df_0p01_R3)

df_omega_0p01_R3.to_csv(f'{path0p01}Delta0p01_alpha0p5_R3_omega.csv')


archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_4.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_5.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_6.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_7.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R4_8.csv']

df_0p01_R4 = cargado_archivos(archivos1)

df_0p01_R4 = convertir_elementos(df_0p01_R4)
#df_0p01_R4.show(3)
df_omega_0p01_R4 = omega(df_0p01_R4)
df_omega_0p01_R4.to_csv(f'{path0p01}Delta0p01_alpha0p5_R4_omega.csv')


archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_4.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_5.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_6.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_7.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R5_8.csv']

df_0p01_R5 = cargado_archivos(archivos1)

df_0p01_R5 = convertir_elementos(df_0p01_R5)
#df_0p01_R5.show(3)
df_omega_0p01_R5 = omega(df_0p01_R5)
df_omega_0p01_R5.to_csv(f'{path0p01}Delta0p01_alpha0p5_R5_omega.csv')

archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_4.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_5.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_6.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_7.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R8_8.csv']

df_0p01_R8 = cargado_archivos(archivos1)

df_0p01_R8 = convertir_elementos(df_0p01_R8)
#df_0p01_R8.show(3)
df_omega_0p01_R8 = omega(df_0p01_R8)
df_omega_0p01_R8.to_csv(f'{path0p01}Delta0p01_alpha0p5_R8_omega.csv')

archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_4.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_5.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_6.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_7.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p5_R10_8.csv']

df_0p01_R10 = cargado_archivos(archivos1)

df_0p01_R10 = convertir_elementos(df_0p01_R10)
#df_0p01_R10.show(3)
df_omega_0p01_R10 = omega(df_0p01_R10)
df_omega_0p01_R10.to_csv(f'{path0p01}Delta0p01_alpha0p5_R10_omega.csv')

In [9]:
archivos1 = [f'{path0p05}analisis_random_Delta0p051.csv',
            f'{path0p05}analisis_random_Delta0p052.csv',
            f'{path0p05}analisis_random_Delta0p053.csv',
            f'{path0p05}analisis_random_Delta0p054.csv',
            f'{path0p05}analisis_random_Delta0p055.csv',
            f'{path0p05}analisis_random_Delta0p056.csv',
            f'{path0p05}analisis_random_Delta0p057.csv',
            f'{path0p05}analisis_random_Delta0p058.csv']

df_0p05_R3 = cargado_archivos(archivos1)

df_0p05_R3 = convertir_elementos(df_0p05_R3)
df_omega_0p05_R3 = omega(df_0p05_R3)
df_omega_0p05_R3.to_csv(f'{path0p05}Delta0p05_alpha0p5_R3_omega.csv')

archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_4.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_5.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_6.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_7.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R4_8.csv']

df_0p05_R4 = cargado_archivos(archivos1)
df_0p05_R4 = convertir_elementos(df_0p05_R4)
df_omega_0p05_R4 = omega(df_0p05_R4)
df_omega_0p05_R4.to_csv(f'{path0p05}Delta0p05_alpha0p5_R4_omega.csv')


archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_4.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_5.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_6.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_7.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R5_8.csv']


df_0p05_R5 = cargado_archivos(archivos1)
df_0p05_R5 = convertir_elementos(df_0p05_R5)
df_omega_0p05_R5 = omega(df_0p05_R5)
df_omega_0p05_R5.to_csv(f'{path0p05}Delta0p05_alpha0p5_R5_omega.csv')


archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_4.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_5.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R8_6.csv']


df_0p05_R8 = cargado_archivos(archivos1)
df_0p05_R8 = convertir_elementos(df_0p05_R8)
df_omega_0p05_R8 = omega(df_0p05_R8)
df_omega_0p05_R8.to_csv(f'{path0p05}Delta0p05_alpha0p5_R8_omega.csv')

archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_4.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_5.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_6.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_7.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p5_R10_8.csv']


df_0p05_R10 = cargado_archivos(archivos1)
df_0p05_R10 = convertir_elementos(df_0p05_R10)
df_omega_0p05_R10 = omega(df_0p05_R10)
df_omega_0p05_R10.to_csv(f'{path0p05}Delta0p05_alpha0p5_R10_omega.csv')

In [10]:
archivos1 = [f'{path0p1}analisis_random_Delta0p11.csv',
            f'{path0p1}analisis_random_Delta0p12.csv',
            f'{path0p1}analisis_random_Delta0p13.csv',
            f'{path0p1}analisis_random_Delta0p14.csv',
            f'{path0p1}analisis_random_Delta0p15.csv',
            f'{path0p1}analisis_random_Delta0p16.csv',
            f'{path0p1}analisis_random_Delta0p17.csv',
            f'{path0p1}analisis_random_Delta0p18.csv']

df_0p1_R3 = cargado_archivos(archivos1)

df_0p1_R3 = convertir_elementos(df_0p1_R3)

df_omega_0p1_R3 = omega(df_0p1_R3)
df_omega_0p1_R3.to_csv(f'{path0p1}Delta0p1_alpha0p5_R3_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_4.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_5.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_6.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_7.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R4_8.csv']

df_0p1_R4 = cargado_archivos(archivos1)

df_0p1_R4 = convertir_elementos(df_0p1_R4)

df_omega_0p1_R4 = omega(df_0p1_R4)
df_omega_0p1_R4.to_csv(f'{path0p1}Delta0p1_alpha0p5_R4_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_4.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_5.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_6.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_7.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R5_8.csv']


df_0p1_R5 = cargado_archivos(archivos1)

df_0p1_R5 = convertir_elementos(df_0p1_R5)
df_omega_0p1_R5 = omega(df_0p1_R5)
df_omega_0p1_R5.to_csv(f'{path0p1}Delta0p1_alpha0p5_R5_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_4.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_5.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R8_6.csv']


df_0p1_R8 = cargado_archivos(archivos1)

df_0p1_R8 = convertir_elementos(df_0p1_R8)
df_omega_0p1_R8 = omega(df_0p1_R8)
df_omega_0p1_R8.to_csv(f'{path0p1}Delta0p1_alpha0p5_R8_omega.csv')


archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_4.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_5.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_6.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_7.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p5_R10_8.csv']


df_0p1_R10 = cargado_archivos(archivos1)

df_0p1_R10 = convertir_elementos(df_0p1_R10)
df_omega_0p1_R10 = omega(df_0p1_R10)
df_omega_0p1_R10.to_csv(f'{path0p1}Delta0p1_alpha0p5_R10_omega.csv')

In [11]:
archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p11.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p12.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p13.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p14.csv']

df_0p01_R3 = cargado_archivos(archivos1)

df_0p01_R3 = convertir_elementos(df_0p01_R3)
df_omega_0p01_R3 = omega(df_0p01_R3)
df_omega_0p01_R3.to_csv(f'{path0p01}Delta0p01_alpha0p1_R3_omega.csv')

archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p1_R4_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R4_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R4_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R4_4.csv']

df_0p01_R4 = cargado_archivos(archivos1)

df_0p01_R4 = convertir_elementos(df_0p01_R4)
df_omega_0p01_R4 = omega(df_0p01_R4)
df_omega_0p01_R4.to_csv(f'{path0p01}Delta0p01_alpha0p1_R4_omega.csv')

archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p1_R5_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R5_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R5_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R5_4.csv']

df_0p01_R5 = cargado_archivos(archivos1)

df_0p01_R5 = convertir_elementos(df_0p01_R5)
df_omega_0p01_R5 = omega(df_0p01_R5)
df_omega_0p01_R5.to_csv(f'{path0p01}Delta0p01_alpha0p1_R5_omega.csv')

archivos1 = [f'{path0p01}analisis_random_Delta0p01_alpha0p1_R10_1.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R10_2.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R10_3.csv',
            f'{path0p01}analisis_random_Delta0p01_alpha0p1_R10_4.csv']


df_0p01_R10 = cargado_archivos(archivos1)

df_0p01_R10 = convertir_elementos(df_0p01_R10)
df_omega_0p01_R10 = omega(df_0p01_R10)
df_omega_0p01_R10.to_csv(f'{path0p01}Delta0p01_alpha0p1_R10_omega.csv')

In [11]:
archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p11.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p12.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p13.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p14.csv']

df_0p05_R3 = cargado_archivos(archivos1)

df_0p05_R3 = convertir_elementos(df_0p05_R3)
df_omega_0p05_R3 = omega(df_0p05_R3)
df_omega_0p05_R3.to_csv(f'{path0p05}Delta0p05_alpha0p1_R3_omega.csv')

archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p1_R4_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R4_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R4_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R4_4.csv']

df_0p05_R4 = cargado_archivos(archivos1)

df_0p05_R4 = convertir_elementos(df_0p05_R4)
df_omega_0p05_R4 = omega(df_0p05_R4)
df_omega_0p05_R4.to_csv(f'{path0p05}Delta0p05_alpha0p1_R4_omega.csv')

archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p1_R5_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R5_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R5_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R5_4.csv']


df_0p05_R5 = cargado_archivos(archivos1)

df_0p05_R5 = convertir_elementos(df_0p05_R5)
df_omega_0p05_R5 = omega(df_0p05_R5)
df_omega_0p05_R5.to_csv(f'{path0p05}Delta0p05_alpha0p1_R5_omega.csv')

archivos1 = [f'{path0p05}analisis_random_Delta0p05_alpha0p1_R10_1.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R10_2.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R10_3.csv',
            f'{path0p05}analisis_random_Delta0p05_alpha0p1_R10_4.csv']


df_0p05_R10 = cargado_archivos(archivos1)

df_0p05_R10 = convertir_elementos(df_0p05_R10)
df_omega_0p05_R10 = omega(df_0p05_R10)
df_omega_0p05_R10.to_csv(f'{path0p05}Delta0p05_alpha0p1_R10_omega.csv')

In [12]:
archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p11.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p12.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p13.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p14.csv']

df_0p1_R3 = cargado_archivos(archivos1)

df_0p1_R3 = convertir_elementos(df_0p1_R3)
df_0p1_R3.show(3)
df_omega_0p1_R3 = omega(df_0p1_R3)
df_omega_0p1_R3.to_csv(f'{path0p1}Delta0p1_alpha0p1_R3_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p1_R4_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R4_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R4_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R4_4.csv']

df_0p1_R4 = cargado_archivos(archivos1)

df_0p1_R4 = convertir_elementos(df_0p1_R4)

df_omega_0p1_R4 = omega(df_0p1_R4)
df_omega_0p1_R4.to_csv(f'{path0p1}Delta0p1_alpha0p1_R4_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p1_R5_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R5_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R5_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R5_4.csv']


df_0p1_R5 = cargado_archivos(archivos1)

df_0p1_R5 = convertir_elementos(df_0p1_R5)
df_omega_0p1_R5 = omega(df_0p1_R5)
df_omega_0p1_R5.to_csv(f'{path0p1}Delta0p1_alpha0p1_R5_omega.csv')

archivos1 = [f'{path0p1}analisis_random_Delta0p1_alpha0p1_R10_1.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R10_2.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R10_3.csv',
            f'{path0p1}analisis_random_Delta0p1_alpha0p1_R10_4.csv']


df_0p1_R10 = cargado_archivos(archivos1)

df_0p1_R10 = convertir_elementos(df_0p1_R10)
df_omega_0p1_R10 = omega(df_0p1_R10)
df_omega_0p1_R10.to_csv(f'{path0p1}Delta0p1_alpha0p1_R10_omega.csv')

MAp,Ms2,Mx1,st,gX,epsilon,yf,Mx2,Densidad reliquia,loglikelihood,delta,alphad,alphaf,mass_relative
5.67798,2.46971,1.89266,0.0001,1.121,0.000739579,0.0373666,2.08193,4.91e-05,7.19411,0.189266,0.1,0.000111111,0.1
48.9794,1.43402,16.3265,0.0001,1.121,2.18072e-05,0.0373666,17.9591,9.45e-05,7.18866,1.63265,0.1,0.000111111,0.1
86.9118,57.0554,28.9706,0.0001,1.121,0.00087436,0.0373666,31.8677,0.00162,7.00682,2.89706,0.1,0.000111111,0.1
